<a href="https://colab.research.google.com/github/TurkuNLP/textual-data-analysis-course/blob/main/model_explainability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model explainability example

* The new code related to model explainability starts half-way through, after the classifier has been trained


# Train BERT text classifier using the Transformers library

This notebook shows how to train a simple text classifier by fine-tuning a pre-trained BERT model using the Hugging Face [Transformers](https://huggingface.co/transformers/) library.

This notebook is based in part on the [Text classification on GLUE](https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/text_classification.ipynb) notebook.

**NOTE**: it's recommended to run this using a runtime with a GPU. Select "Runtime" -> "Change runtime type" from the top menu in Colab and set "Hardware accelerator" to "GPU" when starting

## Install libraries

First, we'll use [`pip`](https://pypi.org/project/pip/) to install two Python libraries that are used in this notebook: [`transformers`](https://huggingface.co/transformers/) and [`datasets`](https://huggingface.co/docs/datasets/).

In [1]:
!pip --quiet install transformers
!pip --quiet install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 68.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 15.4 MB/s eta 0:00:00


In [2]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer
from datasets import load_dataset

## Set model, dataset and hyperparameters

Let's then set some global variables such as the name of the pre-trained model and the hyperparameters we'll use for fine-tuning it.

* `MODEL_NAME`: the name of a pretrained model included in the [model repository](https://huggingface.co/models)
* `DATASET`: the path and name of a dataset included in the [dataset repository](https://huggingface.co/datasets)
* `LEARNING_RATE`, `BATCH_SIZE`, and `TRAIN_EPOCHS`: hyperparameters to use for fine-turning the model. (Try different values here!)

Here, we'll use the [Stanford Sentiment Treebank](https://huggingface.co/datasets/sst) dataset as prepared for the [GLUE](https://huggingface.co/datasets/glue) collection. This is a binary text classification task where the objective is to determine if sentences express a positive or negative sentiment.

In [3]:
MODEL_NAME = 'bert-base-cased'
DATASET = ('glue', 'sst2')

## Load dataset

We'll first load the dataset with [`load_dataset`](https://huggingface.co/docs/datasets/package_reference/loading_methods.html#datasets.load_dataset) and determine the number of distinct labels in the (training) data.

In [4]:
dataset = load_dataset(*DATASET)
num_labels = len(set(dataset['train']['label']))

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

The loaded dataset is a simple dictionary-like container for distinct `DataSet` objects for training, development (validation), and test data:

In [5]:
print(dataset)
print(f'number of distinct labels: {num_labels}')

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})
number of distinct labels: 2


To reduce training time a bit, let's just take every 10th item in the `train` subset of the dataset.

In [7]:
# I leave this undone to get the best possible model later for the explainability part
#dataset['train'] = dataset['train'].filter(lambda example, idx: idx % 10 == 0, with_indices=True)

## Load tokenizer and tokenize data

We'll then load an appropriate tokenizer for the pre-trained model we'll be using with [`AutoTokenizer.from_pretrained`](https://huggingface.co/transformers/model_doc/auto.html#transformers.AutoTokenizer.from_pretrained)

In [6]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

The tokenizer can take a string (or e.g. a list of strings) and performs all the necessary preprocessing steps to prepare its input for use by the model, including splitting input texts into tokens, adding padding and special characters, and mapping those to integer IDs (`input_ids`). The tokenizer also creates attention weights (`attention_mask`) for the model.

We'll then define an encoding function applying the tokenizer to the text data  (key `"sentence"`) of a [`Dataset`](https://huggingface.co/docs/datasets/package_reference/main_classes.html?highlight=dataset#datasets.Dataset) object, and use the `map` function of the `DatasetDict` to tokenize the train, development, and test datasets.

In [7]:
def encode_dataset(d):
  return tokenizer(d['sentence'])

encoded_dataset = dataset.map(encode_dataset)

  0%|          | 0/67349 [00:00<?, ?ex/s]

  0%|          | 0/872 [00:00<?, ?ex/s]

  0%|          | 0/1821 [00:00<?, ?ex/s]

## Load pre-trained model

Next, we'll load the pre-trained model with support for text classification output using [`AutoModelForSequenceClassification.from_prertained`](https://huggingface.co/transformers/model_doc/auto.html#transformers.AutoModelForSequenceClassification.from_pretrained).

Note that we need to provide the number of labels in the data when loading the model.

In [8]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=num_labels)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

## Training parameters and metrics
We're almost ready to train. We'll next create a [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments) object to hold the hyperparameters and other settings that are appropriate for training on Colab:

* `save_strategy`: set to `"epoch"` so that model checkpoints are saved every epoch, this I can do because I save to own Google Drive
* `evaluation_strategy` and `logging_strategy` set to `"epoch"` so that evaluation and logging are performed once per epoch
* The hyperparameters `LEARNING_RATE`, `BATCH_SIZE` and `TRAIN_EPOCHS` set above are passed to the training process through this object

Finally, we'll define a simple accuracy metric measuring how many predictions match their correct values.

In [9]:
def compute_accuracy(pred):
    y_pred = pred.predictions.argmax(axis=1)
    y_true = pred.label_ids
    return { 'accuracy': sum(y_pred == y_true) / len(y_true) }

# Training (fine-tuning)

For fine-tuning the pre-trained model, we'll create a [`Trainer`](https://huggingface.co/transformers/main_classes/trainer.html) object, providing it with the pre-trained model, settings, training and development (validation) data, and the evaluation metric created above.

In [10]:


# Hyperparameters
LEARNING_RATE=1e-4
BATCH_SIZE=128
TRAIN_EPOCHS=4

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
train_args = TrainingArguments(
    '/content/drive/MyDrive/WorkStuff/ssd2-sentiment-model',    # output directory for checkpoints and predictions
    load_best_model_at_end=True,
    evaluation_strategy='steps',
    logging_strategy='steps',
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    num_train_epochs=TRAIN_EPOCHS,
)

In [14]:
trainer = Trainer(
      model,
      train_args,
      train_dataset=encoded_dataset['train'],
      eval_dataset=encoded_dataset['validation'],
      tokenizer=tokenizer,
      compute_metrics=compute_accuracy
)

Training is then performed simply by calling the `train` function of the `Trainer` object.

In [15]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence, idx. If sentence, idx are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 67349
  Num Epochs = 4
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 2108
  Number of trainable parameters = 108311810
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is fast

Step,Training Loss,Validation Loss,Accuracy
500,0.228600,0.213508,0.915138
1000,0.114100,0.243954,0.908257
1500,0.066500,0.278302,0.913991
2000,0.034800,0.301998,0.912844


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence, idx. If sentence, idx are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 872
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/WorkStuff/ssd2-sentiment-model/checkpoint-500
Configuration saved in /content/drive/MyDrive/WorkStuff/ssd2-sentiment-model/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/WorkStuff/ssd2-sentiment-model/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/WorkStuff/ssd2-sentiment-model/checkpoint-500/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/WorkStuff/ssd2-sentiment-model/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceCl

TrainOutput(global_step=2108, training_loss=0.1070027375130771, metrics={'train_runtime': 2180.2355, 'train_samples_per_second': 123.563, 'train_steps_per_second': 0.967, 'total_flos': 7154743470078660.0, 'train_loss': 0.1070027375130771, 'epoch': 4.0})

## Evaluation and predictions for user input

We can use `trainer.evaluate()` to evaluate the trained model on the `eval_dataset` given to the trainer:

In [16]:
results = trainer.evaluate()
print(f'Accuracy: {results["eval_accuracy"]}')

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence, idx. If sentence, idx are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 872
  Batch size = 8


Accuracy: 0.9151376146788991


# Model save

* Now that the model is trained, we may want to save it to Google Drive
* That way we don't need to train it every time

In [17]:
import torch
#make sure you make this directory
torch.save(trainer.model,"/content/drive/MyDrive/WorkStuff/sent_model.pt")

# Model load

* Now we load the model and repeat some of the imports so it is possible to run the notebook from this point onwards

In [18]:
# relevant stuff repeated from above so you can run it from here onwards if you happen to have the model trained

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer
from datasets import load_dataset

MODEL_NAME = 'bert-base-cased'
DATASET = ('glue', 'sst2')

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/5532cc56f74641d4bb33641f5c76a55d11f846e0/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/5532cc56f74641d4bb33641f5c76a55d11f846e0/vocab.txt
loading file tokenize

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
import torch
model = torch.load("/content/drive/MyDrive/WorkStuff/sent_model.pt")
model.to('cpu')    # simplifies input placement

sentiment_label = [
    'negative',
    'positive'
]

def predict_sentiment(string):
    tokenized = tokenizer(string, return_tensors='pt')
    pred = model(**tokenized)
    pred_idx = pred.logits.detach().numpy().argmax()
    return sentiment_label[pred_idx]

try that out

In [21]:
example_sentences = [
    'I think this is very good, and I am in support.',
    'This is bad news.',
    'This could not have been better.',
    'This is not good news at all.',
    'Donald Trump lost the election and will no longer be every day on CNN.',
    'I think your service fulfilled my expectations, I could not really expect you to be any better'
]

for e in example_sentences:
    print(e, '->', predict_sentiment(e))

I think this is very good, and I am in support. -> positive
This is bad news. -> negative
This could not have been better. -> negative
This is not good news at all. -> negative
Donald Trump lost the election and will no longer be every day on CNN. -> negative
I think your service fulfilled my expectations, I could not really expect you to be any better -> positive


# Model explainability with the captum library

* captum.ai is a nifty library with many of the explainability algorithms implemented
* we will use it here to try the techniques
* this is not the easiest of code, btw, many of the libraries are bleeding-edge, so you cut yourself here and there (the blood on the edge is yours)
* the code below is (quite substantial) adaptation of https://captum.ai/tutorials/Bert_SQUAD_Interpret

In [22]:
!pip install captum pandas matplotlib seaborn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.2 MB/s eta 0:00:00


In [23]:
from captum.attr import visualization as viz
from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer

In [24]:
#Tells the model that it is in evaluation mode, and zeroes out the gradients
model.eval()
model.zero_grad()

# Helper functions

In [25]:
# Forward on the model -> data in, prediction out, nothing fancy really
def predict(inputs, token_type_ids=None, position_ids=None, attention_mask=None):
    pred=model(inputs, token_type_ids=token_type_ids,
                 position_ids=position_ids, attention_mask=attention_mask)
    return pred.logits #return the output of the classification layer

ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
cls_token_id = tokenizer.cls_token_id # A token used for prepending to the concatenated question-text word sequence


In [26]:
# Given input text, construct a pair of (text input, blank reference input as long as the text itself)
# ...token indices:
def construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id, device):
    text_ids = tokenizer.encode(text, add_special_tokens=False)

    # construct input token ids
    input_ids = [cls_token_id] + text_ids + [sep_token_id] #the standard way of feeding the input in

    # construct reference token ids 
    ref_input_ids = [cls_token_id] + [ref_token_id] * len(text_ids) + [sep_token_id]  #basically [CLS] [PAD] [PAD] [PAD] ... [SEP] ... blank
    return torch.tensor([input_ids], device=device), torch.tensor([ref_input_ids], device=device)

# ...token types - since we only have one sentence, these are always 0
def construct_input_ref_token_type_pair(input_ids, device):
    seq_len = input_ids.size(1)
    token_type_ids = torch.zeros((1,seq_len), dtype=torch.long, device=device)
    ref_token_type_ids = torch.zeros_like(token_type_ids, device=device)
    return token_type_ids, ref_token_type_ids

# ...token positions
def construct_input_ref_pos_id_pair(input_ids, device):
    seq_length = input_ids.size(1)
    position_ids = torch.arange(seq_length, dtype=torch.long, device=device)
    # we could potentially also use random permutation with `torch.randperm(seq_length, device=device)`
    ref_position_ids = torch.zeros(seq_length, dtype=torch.long, device=device)

    #make sure shapes match
    position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
    ref_position_ids = ref_position_ids.unsqueeze(0).expand_as(input_ids)
    return position_ids, ref_position_ids

# ...attention mask, that is the same for both input and reference and basically all ones
def construct_attention_mask(input_ids,device):
    return torch.ones_like(input_ids,device=device)



In [27]:
# Let's try it!

device=model.device

text="This piece of news is great, even if you think it is rather unfortunate"

#input:
input_ids, ref_input_ids = construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id, device)
#token type:
token_type_ids, ref_token_type_ids = construct_input_ref_token_type_pair(input_ids, device)
#position ids:
position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids, device)
#attention mask:
attention_mask = construct_attention_mask(input_ids, device)

all_tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
print(all_tokens)

['[CLS]', 'This', 'piece', 'of', 'news', 'is', 'great', ',', 'even', 'if', 'you', 'think', 'it', 'is', 'rather', 'unfortunate', '[SEP]']


In [28]:
p=predict(input_ids,token_type_ids=token_type_ids,position_ids=position_ids,attention_mask=attention_mask)
print("p=",p)
print("p.shape",p.shape)

p= tensor([[-2.2030,  2.0866]], grad_fn=<AddmmBackward0>)
p.shape torch.Size([1, 2])


In [29]:
# Yay, now we finally made it to the attribution part
lig = LayerIntegratedGradients(predict, model.bert.embeddings) #attribute the output wrt to embeddings

In [30]:
# inputs: inputs
# baselines: the blank baseline
# target: which of the two classes in the output (pos/neg) to run the prediction against?
attrs, delta = lig.attribute(inputs=(input_ids,token_type_ids,position_ids,attention_mask),
                                  baselines=(ref_input_ids,ref_token_type_ids,ref_position_ids,attention_mask),
                                  return_convergence_delta=True,target=1)
print("attrs shape",attrs.shape)

attrs shape torch.Size([1, 17, 768])


In [31]:
def summarize_attributions(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    return attributions

attrs_sum = summarize_attributions(attrs)
print("attrs_sum shape",attrs_sum.shape)

attrs_sum shape torch.Size([17])


In [32]:
print(attrs_sum)
print(tokenizer.convert_ids_to_tokens(input_ids[0]))

for a,t in zip(attrs_sum,tokenizer.convert_ids_to_tokens(input_ids[0])):
    print(float(a),t)

tensor([ 0.0000, -0.0083,  0.0504,  0.0288, -0.0253,  0.0322,  0.9766,  0.0659,
         0.1446,  0.0355,  0.0662,  0.0211,  0.0176,  0.0266,  0.0012, -0.0927,
        -0.0103], dtype=torch.float64)
['[CLS]', 'This', 'piece', 'of', 'news', 'is', 'great', ',', 'even', 'if', 'you', 'think', 'it', 'is', 'rather', 'unfortunate', '[SEP]']
0.0 [CLS]
-0.008326209618540225 This
0.050408794373007264 piece
0.028807009806395893 of
-0.025323013312930038 news
0.03216457175092813 is
0.9766395873823104 great
0.0658838596415937 ,
0.14461769458044194 even
0.03546381027764635 if
0.06616236834180936 you
0.02108465053993667 think
0.01764321068152354 it
0.026600932870745487 is
0.0011796511562606681 rather
-0.09272094091133058 unfortunate
-0.010342374741475839 [SEP]


Damn, that seems to work!

In [33]:
import captum
from IPython.core.display import HTML, display
x=captum.attr.visualization.format_word_importances(tokenizer.convert_ids_to_tokens(input_ids[0]),attrs_sum)
HTML(x)

# Almost there...

* Let's wrap this all into a function

In [34]:


def predict_and_explain(model,text):
    model.zero_grad() #to be safe perhaps it's not needed
    device=model.device
    #input:
    input_ids, ref_input_ids = construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id, device)
    #token type:
    token_type_ids, ref_token_type_ids = construct_input_ref_token_type_pair(input_ids, device)
    #position ids:
    position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids, device)
    #attention mask:
    attention_mask = construct_attention_mask(input_ids, device)

    all_tokens = tokenizer.convert_ids_to_tokens(input_ids[0])

    lig = LayerIntegratedGradients(predict, model.bert.embeddings)
    prediction=predict(input_ids,token_type_ids,position_ids,attention_mask)[0]
    prediction_cls=int(torch.argmax(prediction))
    print("Prediction:", ("negative","positive")[prediction_cls],"Weights:",prediction.tolist())
    for target,classname in enumerate(("negative","positive")):
        
        attrs, delta = lig.attribute(inputs=(input_ids,token_type_ids,position_ids,attention_mask),
                                  baselines=(ref_input_ids,ref_token_type_ids,ref_position_ids,attention_mask),
                                  return_convergence_delta=True,target=target)
        attrs_sum = summarize_attributions(attrs)

        x=captum.attr.visualization.format_word_importances(all_tokens,attrs_sum)
        print("ATTRIBUTION WITH RESPECT TO",classname)
        display(HTML(x))
        print()

In [37]:
predict_and_explain(model,"I like ChatGPT. It is an awesome technology, even though I see some dangerous aspects too.")

Prediction: positive Weights: [-2.3010997772216797, 2.283381938934326]
ATTRIBUTION WITH RESPECT TO negative



ATTRIBUTION WITH RESPECT TO positive
